# This is a notebook for querying the merged TS instance for the 1.0.0 Beacon release.

-   You can run each cell individually by pressing "shift + enter".
-   For more information, questions, bugs, please contact us on Slack:
    -   https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.


In [ ]:
Token = ""

#### Install the following packages, if you have not already installed them in your environment:

-   pip install requests
-   pip install xarray
-   pip install ipywidgets
-   pip install cartopy
-   pip install h5netcdf
-   pip install netcdf4
-   pip install scipy
-   pip install packaging


#### Import the required packages


In [39]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Retrieve the available columns from the Merged TS endpoint

Swagger page here: https://beacon-wb1-ts.maris.nl/swagger/


In [40]:
responseinfo = requests.get(
    "https://beacon-wb1-ts.maris.nl/api/query/available-columns",
    headers={"Authorization": f"Bearer {Token}"},
)


params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").


In [41]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("temp")  # Enter your search term here

Matching columns:
TEMP
TEMP_QC
TEMP.standard_name
TEMP.units
COMMON_TEMPERATURE
COMMON_TEMPERATURE_QC
COMMON_TEMPERATURE_STANDARD_NAME
COMMON_TEMPERATURE_UNITS
COMMON_TEMPERATURE_P01
COMMON_TEMPERATURE_P06
COMMON_TEMPERATURE_L05
COMMON_TEMPERATURE_L06
COMMON_TEMPERATURE_L22
COMMON_TEMPERATURE_L35
TEMPPR01
TEMPPR01_SEADATANET_QC
TEMPPR01.units
TEMPPR01.long_name
TEMPPR01.sdn_parameter_urn
TEMPPR01.sdn_uom_urn
Temperature
Temperature.units
Temperature.standard_name
Temperature_WODflag
TEMP_ADJUSTED
TEMP_ADJUSTED_QC


#### You can define here your input parameters


In [42]:
# parameter = "COMMON_TEMPERATURE" #column name
mindate = "2003-01-01"  # yyyy-mm-dd
maxdate = "2005-01-01"  # yyyy-mm-dd
minlon = -8
maxlon = 36.5
minlat = 30
maxlat = 46
AOI_POLY = [
    [-9.50, 30.00],
    [-9.50, 39.00],
    [-0.80, 43.20],
    [16.00, 46.50],
    [36.50, 38.00],
    [36.50, 30.00],
    [-9.50, 30.00]
]

#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs.

-   For more query examples and explanations, you can take a look at https://github.com/maris-development/beacon-blue-cloud.


In [ ]:
def query(mindate, maxdate, minlon, maxlon, minlat, maxlat):
    body = {
        "query_parameters": [
            {
                "column_name": "COMMON_TEMPERATURE",
                "alias": "TEMPERATURE",
            },
            {
                "column_name": "COMMON_TEMPERATURE_QC",
                "alias": "TEMPERATURE_QC",
            },
            {
                "column_name": "COMMON_TEMPERATURE_P01",
                "alias": "TEMPERATURE_P01",
            },
            {
                "column_name": "COMMON_TEMPERATURE_P06",
                "alias": "TEMPERATURE_P06",
            },
            {
                "column_name": "COMMON_SALINITY",
                "alias": "SALINITY",
            },
            {
                "column_name": "COMMON_SALINITY_QC",
                "alias": "SALINITY_QC",
            },
            {
                "column_name": "COMMON_SALINITY_P01",
                "alias": "SALINITY_P01",
            },
            {
                "column_name": "COMMON_SALINITY_P06",
                "alias": "SALINITY_P06",
            },
            {"column_name": "COMMON_TIME", "alias": "time_ISO8601"},
            {"column_name": "COMMON_DEPTH", "alias": "Depth [m]"},
            {
                "column_name": "COMMON_DEPTH_QC",
                "alias": "DEPTH_QC",
            },
            {
                "column_name": "COMMON_DEPTH_P01",
                "alias": "DEPTH_P01",
            },
            {
                "column_name": "COMMON_DEPTH_P06",
                "alias": "DEPTH_P06",
            },
            {"column_name": "COMMON_LONGITUDE", "alias": "LONGITUDE"},
            {"column_name": "COMMON_LATITUDE", "alias": "LATITUDE"},
            {
                "column_name": "COMMON_EDMO_CODE",
                "alias": "EDMO_CODE",
            },
            {"column_name": "SOURCE_BDI", "alias": "SOURCE_BDI"},
            {"column_name": "SOURCE_BDI_DATASET_ID",
                "alias": "SOURCE_BDI_DATASET_ID"},
            {"column_name": "COMMON_ODV_TAG"},
            {"column_name": "PLATFORM_NUMBER"},
        ],
        "filters": [
            {
                "for_query_parameter": "datetime",
                "min": mindate,
                "max": maxdate,
            },
            {
                "longitude_column": "LONGITUDE",
                "latitude_column": "LATITUDE",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [AOI_POLY],
                }
            }
        ],
        "output": {
            "format": {
                "odv": {
                    "longitude_column": {"column_name": "LONGITUDE"},
                    "latitude_column": {"column_name": "LATITUDE"},
                    "time_column": "time_ISO8601",
                    "depth_column": {
                        "column_name": "Depth [m]",
                        "qf_column": "DEPTH_QC",
                    },
                    "data_columns": [
                        {
                            "column_name": "TEMPERATURE",
                            "qf_column": "TEMPERATURE_QC",
                        },
                        {
                            "column_name": "SALINITY",
                            "qf_column": "SALINITY_QC",
                        },
                    ],
                    "metadata_columns": [
                        {
                            "column_name": "TEMPERATURE_P01",
                        },
                        {
                            "column_name": "TEMPERATURE_P06",
                        },
                        {
                            "column_name": "SALINITY_P01",
                        },
                        {
                            "column_name": "SALINITY_P06",
                        },
                        {
                            "column_name": "DEPTH_P01",
                        },
                        {
                            "column_name": "DEPTH_P06",
                        },
                        {
                            "column_name": "SOURCE_BDI",
                        },
                        {
                            "column_name": "PLATFORM_NUMBER",
                        },
                    ],
                    "qf_schema": "SEADATANET",
                    "key_column": "COMMON_ODV_TAG",
                    # With this line we are asking the zip file to be gzip compressed
                    "archiving": "zip_deflate",
                }
            }
        },
    }
    return body


query_body = query(mindate, maxdate, minlon, maxlon, minlat, maxlat)

#### This is the post request that is sent to Beacon with the above specified body.


In [44]:
response = requests.post(
    "https://beacon-wb1-ts.maris.nl/api/query",
    json.dumps(query_body),
    headers={"Authorization": f"Bearer {Token}",
             "Content-type": "application/json"},
)

if response.status_code == 204:
    print(
        "No data has been found for your query, please update your input fields above and run the notebook again."
    )
elif response.status_code != 200:
    print(response.text)

In [45]:
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/TS_merged_{regionname}_{mindate}-{maxdate}.zip", "wb"
).write(response.content)

68496313